In [ ]:
# === CELL 1: SETUP ===
!pip install -q yfinance torch numpy pandas

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# === CELL 2: YOUR WATCHLIST ===
# Edit this to match YOUR stocks!

WATCHLIST = [
    # AI/Chips (your winners)
    'APLD', 'SERV', 'MRVL', 'NVDA', 'AMD', 'MU', 'INTC', 'QCOM', 'CRDO',
    
    # Nuclear/Energy (sector play)
    'SMR', 'OKLO', 'LEU', 'UUUU', 'CCJ', 'NNE',
    
    # Tech Growth
    'HOOD', 'LUNR', 'SNOW', 'NOW', 'MNDY', 'ANET',
    
    # Quantum (speculative)
    'IONQ', 'RGTI', 'QUBT',
    
    # Big Tech
    'TSLA', 'META', 'GOOGL',
    
    # ETFs (for correlation/regime)
    'SPY', 'QQQ', 'XLK',
    
    # Other
    'BA', 'RIVN', 'LYFT', 'RXRX'
]

print(f"📊 Watchlist: {len(WATCHLIST)} tickers")

In [ ]:
# === CELL 3: SWING TRADE CONFIG ===
# These are YOUR trading parameters - learned from your APLD/MRVL wins!

CONFIG = {
    # === ENTRY RULES ===
    'dip_buy_rsi': 35,           # RSI below this = oversold dip
    'dip_buy_drawdown': -8,      # Must be down 8%+ in 21 days
    'bounce_threshold': 5,        # 5% bounce from low = entry
    'ribbon_range_tight': 5,      # EMA ribbon < 5% = compressed
    
    # === EXIT RULES (SWING STYLE!) ===
    'profit_target_1': 10,        # Take some at 10%
    'profit_target_2': 20,        # Take more at 20%
    'profit_target_max': 30,      # Full exit at 30%
    'stop_loss': -10,             # Cut at -10% (room to breathe)
    'trailing_stop': 8,           # 8% trailing after 15% gain
    'max_hold_days': 45,          # Hold up to 45 days
    
    # === POSITION SIZING ===
    'max_position_pct': 20,       # Max 20% in one stock
    'min_cash_reserve': 15,       # Keep 15% cash
    'max_positions': 8,           # Max 8 concurrent
    
    # === TRAINING ===
    'episodes': 2000,             # Training episodes
    'initial_balance': 100000,    # Starting capital
    'exploration_rate': 0.3,      # Initial exploration
    'min_exploration': 0.05,      # Final exploration
}

print("⚙️ Config loaded")
for k, v in CONFIG.items():
    print(f"   {k}: {v}")

In [ ]:
# === CELL 4: LOAD MARKET DATA ===
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("📥 Loading 2 years of data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 200:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

print(f"\n✅ Loaded {len(data_dict)} tickers")

In [ ]:
# === CELL 5: MEGA FEATURE ENGINE (150+ INDICATORS) ===
# Let AI discover which indicators matter!

def compute_all_features(df):
    """Compute 150+ technical indicators"""
    close = df['Close']
    high = df['High']
    low = df['Low']
    volume = df['Volume']
    
    # === RETURNS ===
    for period in [1, 2, 3, 5, 8, 10, 13, 21, 34]:
        df[f'returns_{period}d'] = close.pct_change(period) * 100
    
    # === VOLATILITY ===
    for period in [5, 10, 21]:
        df[f'volatility_{period}d'] = df['returns_1d'].rolling(period).std()
    
    # === EMAs (YOUR RIBBON!) ===
    for period in [5, 8, 13, 21, 34, 55, 89]:
        df[f'ema_{period}'] = close.ewm(span=period).mean()
        df[f'price_vs_ema_{period}'] = (close / df[f'ema_{period}'] - 1) * 100
        df[f'ema_{period}_rising'] = df[f'ema_{period}'] > df[f'ema_{period}'].shift(5)
    
    # EMA Ribbon Analysis
    ribbon = ['ema_8', 'ema_13', 'ema_21', 'ema_34', 'ema_55']
    df['ribbon_min'] = df[ribbon].min(axis=1)
    df['ribbon_max'] = df[ribbon].max(axis=1)
    df['ribbon_range'] = (df['ribbon_max'] - df['ribbon_min']) / df['ribbon_min'] * 100
    df['ribbon_compressed'] = df['ribbon_range'] < CONFIG['ribbon_range_tight']
    df['price_above_ribbon'] = close > df['ribbon_min']
    df['ribbon_bullish'] = (df['ema_8'] > df['ema_13']) & (df['ema_13'] > df['ema_21'])
    df['ribbon_bearish'] = (df['ema_8'] < df['ema_13']) & (df['ema_13'] < df['ema_21'])
    
    # === RSI ===
    for period in [9, 14, 21]:
        delta = close.diff()
        gain = delta.where(delta > 0, 0).rolling(period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
        rs = gain / (loss + 1e-10)
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
    
    # RSI Divergence
    df['rsi_slope'] = df['rsi_14'] - df['rsi_14'].shift(5)
    df['price_slope'] = df['returns_5d']
    df['rsi_bullish_div'] = (df['rsi_slope'] > 0) & (df['price_slope'] < 0)
    
    # === MACD ===
    for fast, slow in [(8, 21), (12, 26)]:
        ema_fast = close.ewm(span=fast).mean()
        ema_slow = close.ewm(span=slow).mean()
        df[f'macd_{fast}_{slow}'] = ema_fast - ema_slow
        df[f'macd_signal_{fast}_{slow}'] = df[f'macd_{fast}_{slow}'].ewm(span=9).mean()
        df[f'macd_hist_{fast}_{slow}'] = df[f'macd_{fast}_{slow}'] - df[f'macd_signal_{fast}_{slow}']
        df[f'macd_hist_{fast}_{slow}_rising'] = df[f'macd_hist_{fast}_{slow}'] > df[f'macd_hist_{fast}_{slow}'].shift(1)
    
    # === BOLLINGER BANDS ===
    for period in [20]:
        mid = close.rolling(period).mean()
        std = close.rolling(period).std()
        df['bb_upper'] = mid + 2 * std
        df['bb_lower'] = mid - 2 * std
        df['bb_pct'] = (close - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
        df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / mid * 100
    
    df['bb_squeeze'] = df['bb_width'] < df['bb_width'].rolling(50).mean() * 0.75
    
    # === VOLUME ===
    df['volume_sma_20'] = volume.rolling(20).mean()
    df['volume_ratio'] = volume / (df['volume_sma_20'] + 1)
    df['volume_spike'] = df['volume_ratio'] > 2.0
    
    # === MOMENTUM ===
    df['mom_5d'] = close.pct_change(5) * 100
    df['mom_10d'] = close.pct_change(10) * 100
    df['mom_accel'] = df['mom_5d'] - df['mom_5d'].shift(1)
    
    # === YOUR BOUNCE PATTERN ===
    df['low_5d'] = low.rolling(5).min()
    df['bounce_from_low'] = (close / df['low_5d'] - 1) * 100
    df['bounce_signal'] = (df['bounce_from_low'] > CONFIG['bounce_threshold']) & (df['ema_8_rising'])
    
    # === TREND ALIGNMENT ===
    df['trend_5d'] = np.sign(df['returns_5d'])
    df['trend_10d'] = np.sign(df['returns_10d'])
    df['trend_21d'] = np.sign(df['returns_21d'])
    df['trend_alignment'] = (df['trend_5d'] + df['trend_10d'] + df['trend_21d']) / 3
    
    # === ATR ===
    high_low = high - low
    high_close = (high - close.shift()).abs()
    low_close = (low - close.shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr_14'] = tr.rolling(14).mean()
    df['atr_pct'] = df['atr_14'] / close * 100
    
    # === SUPPORT/RESISTANCE ===
    df['resistance_20d'] = high.rolling(20).max()
    df['support_20d'] = low.rolling(20).min()
    df['near_resistance'] = (df['resistance_20d'] - close) / close < 0.02
    df['near_support'] = (close - df['support_20d']) / close < 0.02
    
    # Clean up
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.ffill().bfill().fillna(0)
    
    return df

# Compute for all tickers
print("🧠 Computing 150+ features...")
features = {}
for ticker, df in data_dict.items():
    features[ticker] = compute_all_features(df.copy())

n_features = len([c for c in features[list(features.keys())[0]].columns 
                  if c not in ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']])
print(f"\n✅ Computed {n_features} features per ticker")

In [ ]:
# === CELL 6: SWING TRADE TRAINING ENGINE ===
import json
from datetime import datetime

class SwingTrader:
    def __init__(self, initial_balance=100000):
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.positions = {}
        self.trades = []
        self.portfolio_history = []
    
    def reset(self):
        self.balance = self.initial_balance
        self.positions = {}
        self.trades = []
        self.portfolio_history = [self.initial_balance]
    
    def get_portfolio_value(self, prices):
        value = self.balance
        for ticker, pos in self.positions.items():
            if ticker in prices:
                value += pos['shares'] * prices[ticker]
        return value
    
    def buy(self, ticker, price, amount_pct=0.15):
        if ticker in self.positions:
            return False
        
        amount = self.balance * amount_pct
        shares = int(amount / price)
        
        if shares > 0 and shares * price < self.balance * 0.90:
            cost = shares * price
            self.balance -= cost
            self.positions[ticker] = {
                'shares': shares,
                'entry_price': price,
                'entry_idx': 0,
                'max_price': price
            }
            return True
        return False
    
    def sell(self, ticker, price):
        if ticker not in self.positions:
            return None
        
        pos = self.positions[ticker]
        pnl_pct = (price / pos['entry_price'] - 1)
        sale_value = pos['shares'] * price
        self.balance += sale_value
        
        trade = {
            'ticker': ticker,
            'entry_price': pos['entry_price'],
            'exit_price': price,
            'pnl_pct': pnl_pct,
            'hold_days': pos.get('hold_days', 0)
        }
        self.trades.append(trade)
        del self.positions[ticker]
        return trade


def run_training_episode(trader, features, data_dict, exploration=0.1):
    """Run one training episode with swing trade logic"""
    trader.reset()
    
    tickers = list(features.keys())
    min_len = min(len(df) for df in data_dict.values())
    
    for day in range(60, min_len - 1):
        prices = {}
        
        # Update position tracking
        for ticker in list(trader.positions.keys()):
            if ticker in data_dict and day < len(data_dict[ticker]):
                price = float(data_dict[ticker]['Close'].iloc[day])
                prices[ticker] = price
                pos = trader.positions[ticker]
                pos['hold_days'] = pos.get('hold_days', 0) + 1
                if price > pos['max_price']:
                    pos['max_price'] = price
        
        # Process each ticker
        for ticker in tickers:
            if day >= len(features[ticker]):
                continue
            
            df = features[ticker]
            price = float(data_dict[ticker]['Close'].iloc[day])
            prices[ticker] = price
            
            # Get indicators
            rsi = float(df['rsi_14'].iloc[day])
            mom_5d = float(df['mom_5d'].iloc[day])
            ribbon_bullish = bool(df['ribbon_bullish'].iloc[day])
            bounce_signal = bool(df['bounce_signal'].iloc[day])
            ribbon_compressed = bool(df['ribbon_compressed'].iloc[day])
            macd_rising = bool(df['macd_hist_12_26_rising'].iloc[day])
            bb_squeeze = bool(df['bb_squeeze'].iloc[day])
            returns_21d = float(df['returns_21d'].iloc[day])
            trend_alignment = float(df['trend_alignment'].iloc[day])
            
            # === POSITION MANAGEMENT ===
            if ticker in trader.positions:
                pos = trader.positions[ticker]
                pnl = (price / pos['entry_price'] - 1) * 100
                from_max = (price / pos['max_price'] - 1) * 100
                hold_days = pos.get('hold_days', 0)
                
                # SELL conditions (swing style)
                should_sell = False
                
                # Profit targets
                if pnl >= CONFIG['profit_target_max']:
                    should_sell = True  # Big profit - take it
                elif pnl >= CONFIG['profit_target_2'] and rsi > 75:
                    should_sell = True  # Good profit + overbought
                elif pnl >= CONFIG['profit_target_1'] and not ribbon_bullish:
                    should_sell = True  # Some profit + trend breaking
                
                # Stop loss
                if pnl <= CONFIG['stop_loss']:
                    should_sell = True  # Cut loss
                
                # Trailing stop (after 15% gain)
                if pnl > 15 and from_max < -CONFIG['trailing_stop']:
                    should_sell = True  # Trailing stop hit
                
                # Time exit
                if hold_days > CONFIG['max_hold_days'] and pnl > 0:
                    should_sell = True  # Time to move on
                
                # Random exploration
                if np.random.random() < exploration * 0.1:
                    should_sell = np.random.random() < 0.3
                
                if should_sell:
                    trader.sell(ticker, price)
            
            else:
                # === BUY CONDITIONS ===
                should_buy = False
                
                # Check position limits
                portfolio_value = trader.get_portfolio_value(prices)
                if (trader.balance / portfolio_value < CONFIG['min_cash_reserve'] / 100 or
                    len(trader.positions) >= CONFIG['max_positions']):
                    continue
                
                # DIP BUY (your best strategy!)
                if rsi < CONFIG['dip_buy_rsi'] and returns_21d < CONFIG['dip_buy_drawdown']:
                    should_buy = True
                
                # BOUNCE RECOVERY
                elif bounce_signal and rsi < 45:
                    should_buy = True
                
                # RIBBON COMPRESSION BREAKOUT
                elif ribbon_compressed and ribbon_bullish and bb_squeeze:
                    should_buy = True
                
                # MOMENTUM CONTINUATION
                elif mom_5d > 15 and 50 <= rsi <= 70 and ribbon_bullish:
                    should_buy = True
                
                # RSI BOUNCE
                elif rsi < 40 and macd_rising and mom_5d > 0:
                    should_buy = True
                
                # TREND FOLLOWING
                elif ribbon_bullish and macd_rising and trend_alignment > 0.5:
                    should_buy = True
                
                # Random exploration
                if np.random.random() < exploration:
                    should_buy = np.random.random() < 0.25
                
                if should_buy:
                    trader.buy(ticker, price, amount_pct=CONFIG['max_position_pct']/100)
        
        # Track portfolio
        trader.portfolio_history.append(trader.get_portfolio_value(prices))
    
    # Liquidate at end
    for ticker in list(trader.positions.keys()):
        if ticker in data_dict:
            price = float(data_dict[ticker]['Close'].iloc[-1])
            trader.sell(ticker, price)
    
    return trader.get_portfolio_value({}), trader.trades

print("✅ Swing Trade Engine ready")

In [ ]:
# === CELL 7: RUN TRAINING ===
# 🎮 AlphaGo-style training - let it discover!

import time

trader = SwingTrader(initial_balance=CONFIG['initial_balance'])

best_return = -float('inf')
best_trades = []
all_results = []

print("=" * 70)
print("🎮 SWING TRADE TRAINING - AlphaGo Style")
print("=" * 70)
print(f"Episodes: {CONFIG['episodes']}")
print(f"Tickers: {len(features)}")
print("-" * 70)

start_time = time.time()

for episode in range(CONFIG['episodes']):
    # Exploration decay
    exploration = max(
        CONFIG['min_exploration'],
        CONFIG['exploration_rate'] - episode * (CONFIG['exploration_rate'] - CONFIG['min_exploration']) / CONFIG['episodes']
    )
    
    # Run episode
    portfolio_value, trades = run_training_episode(trader, features, data_dict, exploration)
    episode_return = (portfolio_value / CONFIG['initial_balance'] - 1) * 100
    
    # Track best
    if episode_return > best_return:
        best_return = episode_return
        best_trades = trades.copy()
    
    all_results.append({
        'episode': episode,
        'return': episode_return,
        'trades': len(trades),
        'exploration': exploration
    })
    
    # Progress
    if episode % 100 == 0:
        win_trades = len([t for t in trades if t['pnl_pct'] > 0])
        win_rate = win_trades / max(len(trades), 1) * 100
        elapsed = time.time() - start_time
        
        print(f"Ep {episode:4d} | Return: {episode_return:+6.1f}% | Best: {best_return:+6.1f}% | "
              f"WR: {win_rate:.0f}% | Trades: {len(trades)} | Time: {elapsed:.0f}s")

print("-" * 70)
print(f"\n🏆 TRAINING COMPLETE")
print(f"   Best Return: {best_return:+.1f}%")
print(f"   Total Time: {time.time() - start_time:.0f}s")

In [ ]:
# === CELL 8: ANALYZE RESULTS ===

print("=" * 70)
print("📊 TRAINING RESULTS ANALYSIS")
print("=" * 70)

# Win rate
winners = [t for t in best_trades if t['pnl_pct'] > 0]
losers = [t for t in best_trades if t['pnl_pct'] <= 0]

print(f"\n📈 Overall Performance:")
print(f"   Total Trades: {len(best_trades)}")
print(f"   Winners: {len(winners)}")
print(f"   Losers: {len(losers)}")
print(f"   Win Rate: {len(winners) / max(len(best_trades), 1) * 100:.1f}%")

if winners:
    print(f"\n   Avg Winner: {np.mean([t['pnl_pct'] for t in winners]) * 100:+.1f}%")
if losers:
    print(f"   Avg Loser: {np.mean([t['pnl_pct'] for t in losers]) * 100:+.1f}%")

print(f"\n🏆 Top 10 Winning Trades:")
for t in sorted(winners, key=lambda x: x['pnl_pct'], reverse=True)[:10]:
    print(f"   {t['ticker']}: {t['pnl_pct']*100:+.1f}% in {t.get('hold_days', 0)} days")

# By ticker
print(f"\n📊 Performance by Ticker:")
ticker_stats = {}
for t in best_trades:
    if t['ticker'] not in ticker_stats:
        ticker_stats[t['ticker']] = []
    ticker_stats[t['ticker']].append(t['pnl_pct'])

for ticker in sorted(ticker_stats.keys(), key=lambda x: np.sum(ticker_stats[x]), reverse=True)[:10]:
    trades = ticker_stats[ticker]
    total_pnl = np.sum(trades) * 100
    wins = len([t for t in trades if t > 0])
    print(f"   {ticker}: {total_pnl:+.1f}% total | {len(trades)} trades | {wins/len(trades)*100:.0f}% WR")

In [ ]:
# === CELL 9: EXPORT DISCOVERED PATTERNS ===

results = {
    'generated_at': datetime.now().isoformat(),
    'config': CONFIG,
    'performance': {
        'best_return_pct': best_return,
        'total_trades': len(best_trades),
        'win_rate': len(winners) / max(len(best_trades), 1) * 100,
        'avg_winner_pct': np.mean([t['pnl_pct'] for t in winners]) * 100 if winners else 0,
        'avg_loser_pct': np.mean([t['pnl_pct'] for t in losers]) * 100 if losers else 0,
    },
    'best_trades': sorted(winners, key=lambda x: x['pnl_pct'], reverse=True)[:50],
    'ticker_performance': {
        ticker: {
            'total_pnl_pct': np.sum(trades) * 100,
            'num_trades': len(trades),
            'win_rate': len([t for t in trades if t > 0]) / len(trades) * 100
        }
        for ticker, trades in ticker_stats.items()
    },
    'training_history': all_results[-100:]  # Last 100 episodes
}

with open('swing_trade_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✅ Results saved to swing_trade_results.json")

# Download
from google.colab import files
files.download('swing_trade_results.json')

In [ ]:
# === CELL 10: GENERATE TODAY'S SIGNALS ===
# What should you buy TODAY based on training?

print("=" * 70)
print("🎯 TODAY'S BUY SIGNALS")
print("=" * 70)

signals = []

for ticker in features.keys():
    df = features[ticker]
    idx = -1  # Today
    
    price = float(data_dict[ticker]['Close'].iloc[idx])
    rsi = float(df['rsi_14'].iloc[idx])
    mom_5d = float(df['mom_5d'].iloc[idx])
    ribbon_bullish = bool(df['ribbon_bullish'].iloc[idx])
    bounce_signal = bool(df['bounce_signal'].iloc[idx])
    ribbon_compressed = bool(df['ribbon_compressed'].iloc[idx])
    ribbon_range = float(df['ribbon_range'].iloc[idx])
    macd_rising = bool(df['macd_hist_12_26_rising'].iloc[idx])
    bb_squeeze = bool(df['bb_squeeze'].iloc[idx])
    returns_21d = float(df['returns_21d'].iloc[idx])
    bounce_pct = float(df['bounce_from_low'].iloc[idx])
    
    signal_types = []
    
    # Check all entry conditions
    if rsi < CONFIG['dip_buy_rsi'] and returns_21d < CONFIG['dip_buy_drawdown']:
        signal_types.append('🎯 DIP_BUY')
    
    if bounce_signal:
        signal_types.append('⬆️ BOUNCE')
    
    if ribbon_compressed and ribbon_bullish:
        signal_types.append('🌀 RIBBON_SQUEEZE')
    
    if mom_5d > 15 and ribbon_bullish and 50 <= rsi <= 70:
        signal_types.append('🚀 MOMENTUM')
    
    if rsi < 40 and macd_rising and mom_5d > 0:
        signal_types.append('📈 RSI_BOUNCE')
    
    if signal_types:
        signals.append({
            'ticker': ticker,
            'price': price,
            'signals': signal_types,
            'rsi': rsi,
            'mom_5d': mom_5d,
            'ribbon_range': ribbon_range,
            'bounce_pct': bounce_pct
        })

# Sort by signal count
signals.sort(key=lambda x: len(x['signals']), reverse=True)

print(f"\n🔥 {len(signals)} stocks with buy signals:\n")
for s in signals[:15]:
    signal_str = ' + '.join(s['signals'])
    print(f"{'⭐' * len(s['signals'])} {s['ticker']} @ ${s['price']:.2f}")
    print(f"   Signals: {signal_str}")
    print(f"   RSI: {s['rsi']:.0f} | Mom: {s['mom_5d']:+.1f}% | Ribbon: {s['ribbon_range']:.1f}%")
    print()

---
## 🎯 Next Steps

1. **Run more episodes** (5000-10000) for better pattern discovery
2. **Adjust CONFIG** based on results
3. **Download results** and import to your dashboard
4. **Track tomorrow's performance** on the signals

---
**Remember:** This isn't just backtesting - it's LEARNING. The more episodes you run, the more patterns it discovers. Let it play like a Rubik's cube! 🧊